# DiffusionGS Colab Playbook

This notebook consolidates the data prep, configuration, training, and evaluation steps for running DiffusionGS in Google Colab. The cells mirror the commands from the README so you can run end-to-end experiments without modifying the source tree.

## Requirements and variables

Set these environment variables/arguments before running the heavy cells:

* `PROJECT_ROOT`: path where the repo is checked out (default: `/content/Open-DiffusionGS`).
* `RAW_DATA_DIR`: folder containing unzipped RealEstate10K zips.
* `PROCESSED_DIR`: output folder for processed RealEstate10K data (`full_list.txt` lives here).
* `OBJ_JSON_DIR`: folder with Objaverse JSON splits (`train.json`, `val.json`, `test.json`).
* `OBJ_IMAGE_DIR`: Objaverse image root (e.g., extracted `gobjaverse` folder).
* `OUTPUT_DIR`: root folder to collect checkpoints, renders, and logs (often a Drive path).
* `HF_TOKEN`: optional Hugging Face token if you host splits/checkpoints privately.

These values can also be exported in Colab via `os.environ[...] = ...` or Colab form fields.

In [ ]:
import os
from pathlib import Path

# Point to your repo checkout; keep the default on Colab.
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", "/content/Open-DiffusionGS"))
RAW_DATA_DIR = Path(os.getenv("RAW_DATA_DIR", "/content/drive/MyDrive/re10k_raw"))
PROCESSED_DIR = Path(os.getenv("PROCESSED_DIR", "/content/drive/MyDrive/re10k_processed"))
OBJ_JSON_DIR = Path(os.getenv("OBJ_JSON_DIR", "/content/drive/MyDrive/gobjaverse/json"))
OBJ_IMAGE_DIR = Path(os.getenv("OBJ_IMAGE_DIR", "/content/drive/MyDrive/gobjaverse"))
OUTPUT_DIR = Path(os.getenv("OUTPUT_DIR", "/content/drive/MyDrive/diffusiongs"))

PROJECT_ROOT.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"RAW_DATA_DIR: {RAW_DATA_DIR}")
print(f"PROCESSED_DIR: {PROCESSED_DIR}")
print(f"OBJ_JSON_DIR: {OBJ_JSON_DIR}")
print(f"OBJ_IMAGE_DIR: {OBJ_IMAGE_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

## Colab quickstart (no code edits)

1. Run the Drive mount cell below.
2. Run the clone/setup cell to fetch the repo into `PROJECT_ROOT`.
3. Run the dependency install cell (first session only).
4. Fill in the dataset paths (or enable the optional download toggles).
5. Choose continuous vs. discrete+octree configs and launch training.
6. Run inference and the FID/KID cell to score results.
7. Save outputs to Drive automatically.

In [ ]:
%%bash
set -e
if [ ! -d "$PROJECT_ROOT/.git" ]; then
  git clone https://github.com/PRISM-LAB/Open-DiffusionGS.git "$PROJECT_ROOT"
else
  echo 'Repo already present at $PROJECT_ROOT'
fi
cd "$PROJECT_ROOT"
git status --short
pwd

## Mount Google Drive (Colab)

Run this once per session to access data and to save outputs to Drive.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted. If your repo is elsewhere, update PROJECT_ROOT accordingly.')
except Exception as exc:  # noqa: BLE001
    print('Not running inside Colab or Drive mount failed:', exc)

## Install dependencies (first run)

Uncomment the `%pip` line the first time you open the notebook to install project requirements in Colab.

In [ ]:
import os
if os.getenv('SKIP_PIP_INSTALL', '0').lower() not in {'1','true'}:
    !pip install -r requirement.txt
else:
    print('Skipping dependency install because SKIP_PIP_INSTALL is set.')

### Confirm working directory
Ensures Python paths point to the cloned repo before running preprocessing or training.

In [ ]:
import os
import sys
from pathlib import Path

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print('CWD ->', Path.cwd())

## RealEstate10K preprocessing

Convert the downloaded RealEstate10K zips into the `train/full_list.txt` and `test/full_list.txt` files expected by the trainer. The commands mirror the README.

In [ ]:
%%bash
cd "$PROJECT_ROOT"
python process_data.py --base_path "$RAW_DATA_DIR" --output_dir "$PROCESSED_DIR" --mode train
python process_data.py --base_path "$RAW_DATA_DIR" --output_dir "$PROCESSED_DIR" --mode test

In [ ]:
from pathlib import Path

train_list = Path(PROCESSED_DIR) / "train/full_list.txt"
test_list = Path(PROCESSED_DIR) / "test/full_list.txt"

for name, path in (('train', train_list), ('test', test_list)):
    if path.exists():
        print(f"{name} list: {path}")
        with path.open() as fp:
            sample = [fp.readline().strip() for _ in range(5)]
        print("Sample entries:", [s for s in sample if s])
    else:
        print(f"Missing {name} list at {path}; ensure preprocessing succeeded.")

## Objaverse splits and assets

Set the JSON splits and image directory. You can symlink existing Drive folders or download public splits if you host them yourself.

In [ ]:
from pathlib import Path

# Toggle to fetch remote JSON splits you host (set OBJ_JSON_URL accordingly).
DOWNLOAD_JSON_SPLITS = False
OBJ_JSON_URL = os.getenv("OBJ_JSON_URL", "")  # e.g., a Hugging Face file URL if you mirror the splits.
LOCAL_JSON_ARCHIVE = OUTPUT_DIR / "objaverse_json.tar.gz"

OBJ_JSON_DIR.mkdir(parents=True, exist_ok=True)
OBJ_IMAGE_DIR.mkdir(parents=True, exist_ok=True)

if DOWNLOAD_JSON_SPLITS and OBJ_JSON_URL:
    !wget -O "$LOCAL_JSON_ARCHIVE" "$OBJ_JSON_URL"
    !tar -xzf "$LOCAL_JSON_ARCHIVE" -C "$OBJ_JSON_DIR"
else:
    print("Skipping download; using existing OBJ_JSON_DIR and OBJ_IMAGE_DIR.")
print("JSON dir:", OBJ_JSON_DIR)
print("Image dir:", OBJ_IMAGE_DIR)

## ShapeNet category subsets (optional)

If you want to fine-tune on specific ShapeNet synsets, set `USE_SHAPENET_CONFIG=True` and provide a category mapping JSON plus a comma-separated category list in `SHAPENET_CATEGORY_FILTER`. You can keep everything in Drive and run entirely from this notebook.


In [ ]:
# Optionally switch to the ShapeNet-specific config and filter by categories
USE_SHAPENET_CONFIG = False  # Set True to start from diffusionGS/configs/diffusionGS_shapenet.yaml
SHAPENET_CATEGORY_MAP = os.getenv("SHAPENET_CATEGORY_MAP", str(PROJECT_ROOT / "examp_data/shapenet_category_map.example.json"))
SHAPENET_CATEGORY_FILTER = os.getenv("SHAPENET_CATEGORY_FILTER", "")
SHAPENET_CATEGORY_KEY = os.getenv("SHAPENET_CATEGORY_KEY", "category")


## Configure datamodule and scheduler options

Flip `DISCRETE_TOKENIZE` / `DISCRETE_OCTREE_DEPTH` before writing the final config. If you want to use the discrete scheduler, set `USE_DISCRETE_SCHEDULER=True` to override `system.noise_scheduler_type`.

In [ ]:
from omegaconf import OmegaConf

default_config = "diffusionGS/configs/diffusionGS_shapenet.yaml" if USE_SHAPENET_CONFIG else "diffusionGS/configs/diffusionGS_rel.yaml"
CONFIG_PATH = os.getenv("CONFIG_PATH", str(PROJECT_ROOT / default_config))
TMP_CONFIG_PATH = str(PROJECT_ROOT / "notebooks/colab_config.yaml")
DISCRETE_TOKENIZE = False
DISCRETE_OCTREE_DEPTH = 3
USE_DISCRETE_SCHEDULER = True

cfg = OmegaConf.load(CONFIG_PATH)

# Point to Objaverse or ShapeNet assets; fall back to notebook paths when placeholders are detected
if not cfg.data.get('local_dir') or 'DATAPATH' in str(cfg.data.local_dir):
    cfg.data.local_dir = str(OBJ_JSON_DIR)
if not cfg.data.get('image_dir') or 'DATAPATH' in str(cfg.data.image_dir):
    cfg.data.image_dir = str(OBJ_IMAGE_DIR)

cfg.data.local_eval_dir = cfg.data.get('local_eval_dir', str(test_list))

# Toggle discrete tokenization
cfg.data.discrete_tokenize = DISCRETE_TOKENIZE
cfg.data.discrete_octree_depth = DISCRETE_OCTREE_DEPTH

# Optional ShapeNet category filtering
shapenet_categories = [c.strip() for c in SHAPENET_CATEGORY_FILTER.split(',') if c.strip()]
if USE_SHAPENET_CONFIG or shapenet_categories:
    cfg.data.category_mapping_json = str(SHAPENET_CATEGORY_MAP)
    cfg.data.category_filter = shapenet_categories or cfg.data.get('category_filter', [])
    cfg.data.category_key = SHAPENET_CATEGORY_KEY

if USE_DISCRETE_SCHEDULER:
    cfg.system.noise_scheduler_type = "diffusionGS.models.scheduler.discrete_scheduler.DiscreteScheduler"

# Save a copy so launch.py can consume it directly
Path(TMP_CONFIG_PATH).parent.mkdir(parents=True, exist_ok=True)
OmegaConf.save(cfg, TMP_CONFIG_PATH)
print("Wrote config to", TMP_CONFIG_PATH)
print(OmegaConf.to_yaml(cfg))


### Toggle continuous vs. octree-token configs

Choose whether to run the discrete octree-tokenized pipeline or fall back to a continuous scheduler without editing the YAML by hand.

In [ ]:
# Generate separate configs for octree tokens and continuous scheduling
RUN_WITH_OCTREE_TOKENS = True  # Set False to compare against the continuous scheduler.

OCTREE_CONFIG_PATH = str(PROJECT_ROOT / 'notebooks/colab_config_octree.yaml')
CONTINUOUS_CONFIG_PATH = str(PROJECT_ROOT / 'notebooks/colab_config_continuous.yaml')

# Preserve the base config overrides set above, then branch into discrete/continuous variants.
base_cfg = OmegaConf.create(OmegaConf.to_container(cfg, resolve=True))

octree_cfg = OmegaConf.create(OmegaConf.to_container(base_cfg, resolve=True))
octree_cfg.data.discrete_tokenize = True
octree_cfg.data.discrete_octree_depth = DISCRETE_OCTREE_DEPTH
octree_cfg.system.noise_scheduler_type = 'diffusionGS.models.scheduler.discrete_scheduler.DiscreteScheduler'
OmegaConf.save(octree_cfg, OCTREE_CONFIG_PATH)

continuous_cfg = OmegaConf.create(OmegaConf.to_container(base_cfg, resolve=True))
continuous_cfg.data.discrete_tokenize = False
continuous_cfg.system.noise_scheduler_type = 'diffusionGS.models.scheduler.ddim_scheduler.DDIMScheduler'
OmegaConf.save(continuous_cfg, CONTINUOUS_CONFIG_PATH)

TMP_CONFIG_PATH = OCTREE_CONFIG_PATH if RUN_WITH_OCTREE_TOKENS else CONTINUOUS_CONFIG_PATH
print('Discrete octree-token config:', OCTREE_CONFIG_PATH)
print('Continuous scheduler config:', CONTINUOUS_CONFIG_PATH)
print('Using config ->', TMP_CONFIG_PATH)


## Initialize Lightning modules

Instantiates the datamodule and system using the saved config. This is useful for checking that dataset paths and scheduler choices resolve correctly before training.

In [ ]:
import diffusionGS
from diffusionGS.utils.config import load_config

cfg_struct = load_config(TMP_CONFIG_PATH, n_gpus=1)

datamodule = diffusionGS.find(cfg_struct.data_type)(cfg_struct.data)
system = diffusionGS.find(cfg_struct.system_type)(cfg_struct.system, resumed=cfg_struct.resume is not None)
system.set_save_dir(str(OUTPUT_DIR / "save"))

print("Datamodule:", datamodule)
print("System:", system.__class__.__name__)

### Optional: pass octree tokens to the scheduler/model

Pulls a small batch to feed `octree_tokens_input` into the scheduler so the diffusion model conditions on the octree tokens when they are available.

In [ ]:
if cfg_struct.data.discrete_tokenize and train_list.exists():
    try:
        sample_batch = next(iter(datamodule.train_dataloader()))
        octree_tokens = sample_batch.get('octree_tokens_input')
        if octree_tokens is None:
            print('Batch missing octree_tokens_input; double-check datamodule settings.')
        else:
            if hasattr(system.noise_scheduler, 'set_conditioning_tokens'):
                system.noise_scheduler.set_conditioning_tokens(octree_tokens)
            print('Octree tokens passed to scheduler/model:', octree_tokens.shape)
    except Exception as exc:
        print('Could not fetch a batch to attach octree tokens:', exc)
elif cfg_struct.data.discrete_tokenize:
    print(f'Skipping token pass-through because {train_list} is missing.')
else:
    print('Discrete tokenization disabled; using the continuous path.')


## Training (stage 1 / stage 2)

Adjust `--gpu` to match your Colab runtime. Stage 2 should reuse the stage-1 checkpoint via `shape_model.pretrained_model_name_or_path` in the config.

In [ ]:
%%bash
cd "$PROJECT_ROOT"
# Stage 1 (e.g., 256px)
python launch.py --config "$TMP_CONFIG_PATH" --gpu 0
# Stage 2 example (update TMP_CONFIG_PATH to point to a stage-2 config with pretrained path set)
# python launch.py --config diffusionGS/configs/diffusionGS_rel_512.yaml --gpu 0

## Inference / evaluation

Render test scenes or objects, then copy outputs to Drive.

In [ ]:
%%bash
cd "$PROJECT_ROOT"
CHECKPOINT_PATH="$OUTPUT_DIR/save/last.ckpt"
RENDER_DIR="$OUTPUT_DIR/renders"
mkdir -p "$RENDER_DIR"
python eval_scene_result.py --path "$CHECKPOINT_PATH" --chunk 64 --out "$RENDER_DIR"

In [ ]:
from pathlib import Path
import shutil

render_dir = OUTPUT_DIR / "renders"
backup_dir = OUTPUT_DIR / "renders_backup"
backup_dir.mkdir(parents=True, exist_ok=True)
for item in render_dir.glob("*"):
    target = backup_dir / item.name
    if item.is_file():
        shutil.copy(item, target)
    elif item.is_dir():
        shutil.copytree(item, target, dirs_exist_ok=True)
print("Copied renders to", backup_dir)

## Compute FID/KID (optional)

After rendering samples, point the metric script at a folder of generated images and a folder of references.

In [ ]:
%%bash
cd "$PROJECT_ROOT"
REFERENCE_DIR="$OUTPUT_DIR/reference_images"  # update to your ground-truth images
GENERATED_DIR="$OUTPUT_DIR/renders"         # generated renders from the inference cell
python scripts/compute_fid_kid.py   --reference "$REFERENCE_DIR"   --generated "$GENERATED_DIR"   --batch-size 8 --image-size 299 --kid-subset-size 100   --output "$OUTPUT_DIR/metrics.json"

## Quick visualization (images or video)

Display a small grid or video preview from the rendered outputs.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

image_paths = sorted((OUTPUT_DIR / "renders").glob("*.png"))[:4]
if image_paths:
    ncols = len(image_paths)
    fig, axes = plt.subplots(1, ncols, figsize=(4 * ncols, 4))
    for ax, img_path in zip(axes, image_paths):
        ax.imshow(Image.open(img_path))
        ax.axis('off')
        ax.set_title(img_path.name)
    plt.show()
else:
    print("No PNG renders found; run the inference cell first.")